# Exploring the NYC Airbnb Market

New York City (NYC) is one of the most-visited cities in the world. As a result, there are many Airbnb listings to meet the high demand for temporary lodging for anywhere between a few nights to many months. In this notebook, we will look at the NYC Airbnb market by combining data from multiple file types like .csv, .tsv, and .xlsx.

The goals are to convert untidy data into appropriate formats to analyze, and answer key questions including:

- What is the average price, per night, of an Airbnb listing in NYC?
- How does the average price of an Airbnb listing, per month, compare to the private rental market?
- How many adverts are for private rooms?
- How do Airbnb listing prices compare across the five NYC boroughs?

## Imports

In [100]:
# Data tools
import pandas as pd
import numpy as np
import datetime as dt

## Data Overview

```airbnb_price.csv``` is a CSV file that includes three columns for each rental location.

| Column | Description |
| ------ | ----------- |
| listing_id | Unique ID for listing |
| price | Price per night |
| nbhood_full | Name of New York City neighborhood |

&nbsp;

```airbnb_room_type.xlsx``` is an Excel file that inclues four columns for each listing. The 'listing_id' column corresponds to the 'listing_id' column in the airbnb_price.csv data.

| Column | Description |
| ------ | ----------- |
|  | Auto-generated row number |
| listing_id | Unique ID for listing |
| room_type | Type of room to rent. Entire home, private room, etc... |
| number_of_reviews | Number of reviews left by happy or disappointed patrons |

&nbsp;

``` airbnb_last_review.tsv``` is a tab-seperated-values file that contains dates for the most recent reviews.

| Column | Description |
| ------ | ----------- |
| listing_id | Unique ID for listing |
| host_name | Name of Airbnb host |
| last_review | Date when the last review was left for the listing |

&nbsp;

**Load data into DataFrames:**

In [101]:
# Load airbnb_price.csv, prices
prices = pd.read_csv('airbnb_price.csv')

# Load airbnb_room_type.xlsx, xls
room_types = pd.read_excel('airbnb_room_type.xlsx', sheet_name='new_york')

# Load airbnb_last_review.tsv, reviews
reviews = pd.read_csv("airbnb_last_review.tsv", sep='\t')

# Print the first five rows of each DataFrame
print(prices.head(), '\n')
print(room_types.head(), '\n')
print(reviews.head())

   listing_id        price                nbhood_full
0        2595  225 dollars         Manhattan, Midtown
1        3831   89 dollars     Brooklyn, Clinton Hill
2        5099  200 dollars     Manhattan, Murray Hill
3        5178   79 dollars  Manhattan, Hell's Kitchen
4        5238  150 dollars       Manhattan, Chinatown 

   Unnamed: 0  listing_id        room_type  number_of_reviews
0         0.0      2595.0  Entire home/apt               48.0
1         1.0      3831.0  Entire home/apt              295.0
2         2.0      5099.0  Entire home/apt               78.0
3         3.0      5121.0     Private room               49.0
4         4.0      5178.0     Private room              454.0 

   listing_id    host_name   last_review
0        2595     Jennifer   May 21 2019
1        3831  LisaRoxanne  July 05 2019
2        5099        Chris  June 22 2019
3        5178     Shunichi  June 24 2019
4        5238          Ben  June 09 2019


## Clean Price data

In [102]:
# Remove ' dollars' from prices column
prices['price'] = prices['price'].str.replace(' dollars', '')

# Convert price column to a number
prices['price'] = prices["price"].astype(int)

print(prices.head())

   listing_id  price                nbhood_full
0        2595    225         Manhattan, Midtown
1        3831     89     Brooklyn, Clinton Hill
2        5099    200     Manhattan, Murray Hill
3        5178     79  Manhattan, Hell's Kitchen
4        5238    150       Manhattan, Chinatown


## Calculate average price

We can calculate the average price for Airbnb rentals, but first let's check to see if there are any outliers in the price column.

In [103]:
prices['price'].sort_values().head(10)

10861     0
10958     0
11577     0
11589     0
11024     0
11025     0
11023     0
24806    10
16643    10
12135    10
Name: price, dtype: int64

In [104]:
prices['price'].sort_values().tail(10)

23601    2500
1234     2500
21838    2600
13528    2990
24432    2995
12508    3000
21731    3613
18430    5000
22085    5100
23916    7500
Name: price, dtype: int64

The maximum price per night is \\$7,500, which is a little crazy, but there are also \\$0 prices too. Let's remove the entries with \\$0 for the price. There are only a few.

In [105]:
# Update prices by removing all free listings from prices
prices = prices[prices['price'] > 0]

# Calculate the average price, avg_price
avg_price = round(prices['price'].sum()/len(prices), 2)

# Print the average price
print("The average price per night for an Airbnb listing in New York City is ${:.2f}.".format(avg_price))

The average price per night for an Airbnb listing in New York City is $141.82.


## Comparing costs to the private rental market

According to Zumper, a 1 bedroom apartment in New York City costs, on average, \\$3,100 per month. Let's convert the per night prices of our listings into monthly costs, so we can compare to the private market.

In [106]:
# Add a new column to the prices DataFrame, price_per_month
prices['price_per_month'] = prices['price'] * 365 / 12

# Calculate average_price_per_month
average_price_per_month = round(prices['price_per_month'].mean(), 2)

# Compare Airbnb and rental market
print("Airbnb monthly costs are ${:,}, while in the private market you would pay {}.".format(average_price_per_month, "$3,100.00"))

Airbnb monthly costs are $4,313.61, while in the private market you would pay $3,100.00.


## Cleaning room type column

Understandably, the monthly cost of Airbnb rooms are more expense than the regular, private market. But let's consider the types of rooms as well.

The ```room_type``` column has several variations for private room listings, specifically:

- "Private room"
- "private room"
- "PRIVATE ROOM"

We can convert these to all lowercase.

In [107]:
# Convert room type to lowercase
room_types['room_type'].str.lower()

# Update the room_type column to category data type
room_types['room_type'] = room_types['room_type'].astype('category')

# Count room type frequencies
room_frequencies = room_types['room_type'].value_counts()

print(room_frequencies)

Entire home/apt    9405
Private room       7752
Shared room         357
Hotel room          100
Name: room_type, dtype: int64


## What timeframe are we working with?

Now let's turn our attention to the reviews DataFrame. The last_review column contains the date of the last review in the format of "Month Day Year" e.g., May 21 2019. We've been asked to find out the earliest and latest review dates in the DataFrame, and ensure the format allows this analysis to be easily conducted going forwards.

In [108]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25209 entries, 0 to 25208
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   listing_id   25209 non-null  int64 
 1   host_name    25201 non-null  object
 2   last_review  25209 non-null  object
dtypes: int64(1), object(2)
memory usage: 591.0+ KB


In [109]:
# Change the data type of the last_review column to datetime
reviews['last_review'] = pd.to_datetime(reviews['last_review'], format="%B %d %Y")

reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25209 entries, 0 to 25208
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   listing_id   25209 non-null  int64         
 1   host_name    25201 non-null  object        
 2   last_review  25209 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 591.0+ KB


In [110]:
# Create first_reviewed, the earliest review date
first_reviewed = reviews["last_review"].dt.date.min()

# Create last_reviewed, the most recent review date
last_reviewed = reviews["last_review"].dt.date.max()

# Print the oldest and newest reviews from the DataFrame
print("The latest Airbnb review is {}, the earliest review is {}".format(last_reviewed, first_reviewed))

The latest Airbnb review is 2019-07-09, the earliest review is 2019-01-01


## Joining the DataFrames

Now we've extracted the information needed, we will merge the three DataFrames to make any future analysis easier to conduct. Once we have joined the data, we will remove any observations with missing values and check for duplicates.

In [111]:
# Merge prices and room_types to create rooms_and_prices
rooms_and_prices = prices.merge(room_types, how='outer', on='listing_id')

# Merge rooms_and_prices with the reviews DataFrame to create airbnb_merged
airbnb_merged = rooms_and_prices.merge(reviews, how='outer', on='listing_id')

# Drop missing values from airbnb_merged
airbnb_merged.dropna(inplace=True)

# Check if there are any duplicate values
print("There are {} duplicates in the DataFrame.".format(airbnb_merged.duplicated().sum()))

There are 0 duplicates in the DataFrame.


## Analyzing prices by neighborhood

Now we have combined all data into a single DataFrame, we will turn our attention to understanding the difference in listing prices between New York City boroughs. We can currently see boroughs listed as the first part of a string within the nbhood_full column, e.g.,

- Manhattan, Midtown
- Brooklyn, Clinton Hill
- Manhattan, Murray Hill
- Manhattan, Hell's Kitchen
- Manhattan, Chinatown

We will therefore need to extract this information from the string and store in a new column, borough, for analysis.

In [114]:
# Extract information from the nbhood_full column and store as a new column, borough
airbnb_merged['borough'] = airbnb_merged['nbhood_full'].str.partition()[0].str.replace(',','')

# Group by borough and calculate summary statistics
boroughs = airbnb_merged.groupby("borough")["price"].agg(["sum", "mean", "median", "count"])

# Round boroughs to 2 decimal places, and sort by mean in descending order
boroughs = boroughs.round(2).sort_values('mean', ascending=False)

print(boroughs)

                sum    mean  median  count
borough                                   
Manhattan  893869.0  171.70   139.0   5206
Brooklyn   742816.0  123.02   100.0   6038
Queens     174429.0   92.73    73.0   1881
Staten      13439.0   86.15    70.5    156
Bronx       30954.0   81.03    63.5    382


# Price range by borough

The above output gives us a summary of prices for listings across the 5 boroughs. In this final task we would like to categorize listings based on whether they fall into specific price ranges, and view this by borough.

We can do this using percentiles and labels to create a new column, price_range, in the DataFrame. Once we have created the labels, we can then group the data and count frequencies for listings in each price range by borough.

We will assign the following categories and price ranges:

| label | price |
| ----- | ----- |
| Budget | \\$0-69 |
| Average | \\$70-175 |
| Expensive | \\$176-350 |
| Extravagant | > \\$350 |

In [129]:
# Create labels for the price range, label_names
label_names = ['Budget', 'Average', 'Expensive', 'Extravagant']

# Create the label ranges, ranges
ranges = [0, 69, 175, 350, np.inf]

# Insert new column, price_range, into DataFrame
airbnb_merged['price_range'] = pd.cut(x=airbnb_merged['price'], bins=ranges, labels=label_names)

# Calculate borough and price_range frequencies, prices_by_borough
prices_by_borough = airbnb_merged.groupby(['borough', 'price_range'])['price_range'].count()
print(prices_by_borough)


borough    price_range
Bronx      Budget          209
           Average         155
           Expensive        14
           Extravagant       4
Brooklyn   Budget         1697
           Average        3324
           Expensive       888
           Extravagant     129
Manhattan  Budget          590
           Average        2868
           Expensive      1456
           Extravagant     292
Queens     Budget          870
           Average         847
           Expensive       145
           Extravagant      19
Staten     Budget           71
           Average          73
           Expensive        12
           Extravagant       0
Name: price_range, dtype: int64
